# Bbacktesting using Backtrader and a single pair

An example how to filter and plot candles for pair data

We will

* Create a trading pair universe consisting of SUSHI-USDC pair

* Create a sample strategy that just logs output

*Note*: This example is complete due to bug in Backtrader plotting

## Creating trading universe

First let's import libraries and initialise our dataset client.

In [1]:

import pandas as pd

from capitalgram.chain import ChainId
from capitalgram.pair import PairUniverse

try:
    import capitalgram
except ImportError:
    !pip install -e git+https://github.com/miohtama/capitalgram-onchain-dex-quant-data.git#egg=capitalgram
    import site
    site.main()

from capitalgram.client import Capitalgram

capitalgram = Capitalgram.create_jupyter_client()

Started Capitalgram in Jupyter notebook environment, configuration is stored in /Users/moo/.capitalgram


Let's create a pair universe for Sushi. [See full example](https://docs.capitalgram.com/examples/pairs.html).
We will create a dataset of 4h candles that trade on Sushiswap on Ethereum.

In [2]:
# Decompress the pair dataset to Python map
columnar_pair_table = capitalgram.fetch_pair_universe()

# Exchange map data is so small it does not need any decompression
exchange_universe = capitalgram.fetch_exchange_universe()

# Convert PyArrow table to Pandas format to continue working on it
all_pairs_dataframe = columnar_pair_table.to_pandas()

# Filter down to pairs that only trade on Sushiswap
sushi_swap = exchange_universe.get_by_name_and_chain(ChainId.ethereum, "sushiswap")
sushi_pairs: pd.DataFrame = all_pairs_dataframe.loc[all_pairs_dataframe['exchange_id'] == sushi_swap.exchange_id]

# Create a Python set of pair ids
wanted_pair_ids = sushi_pairs["pair_id"]

print(f"Sushiswap on Ethereum has {len(wanted_pair_ids)} pairs")

Sushiswap on Ethereum has 1308 pairs


Get daily candles and filter them against our wanted pair set.

In [3]:
from capitalgram.candle import CandleBucket

# Get daily candles as Pandas DataFrame
all_candles = capitalgram.fetch_all_candles(CandleBucket.h24).to_pandas()
sushi_candles = all_candles.loc[all_candles["pair_id"].isin(wanted_pair_ids)]
print(f"Out candle universe size is {len(sushi_candles)} candles")

Out candle universe size is 58627 candles


## Creating the first Backtrader strategy

[See the Backtrader quickstart tutorial](https://www.backtrader.com/docu/quickstart/quickstart/).

In [4]:
import backtrader as bt

class TestStrategy(bt.Strategy):
    """A strategy that just logs closing prices.

    See https://www.backtrader.com/docu/quickstart/quickstart/
    """

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        if self.dataclose[0] < self.dataclose[-1]:
            # current close less than previous close

            if self.dataclose[-1] < self.dataclose[-2]:
                # previous close less than the previous close

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.buy()



## Executing strategy backtest

In [5]:
import datetime

import backtrader.feeds as btfeeds

# Backtrader identifies timestamp column by index
sushi_candles = sushi_candles.set_index(pd.DatetimeIndex(sushi_candles["timestamp"]))
sushi_candles["volume"] = sushi_candles["buy_volume"] + sushi_candles["sell_volume"]

sushi_candles = sushi_candles.loc[sushi_candles['pair_id'] == 7531]

print(f"Feeding {len(sushi_candles)} to Celebro")

# Create a cerebro entity
cerebro = bt.Cerebro(stdstats=False)

# Add a strategy
cerebro.addstrategy(TestStrategy)

# Get a pandas dataframe

# Pass it to the backtrader datafeed and add it to the cerebro

data = bt.feeds.PandasData(dataname=sushi_candles)
cerebro.adddata(data)

Feeding 316 to Celebro


## Running the strategy

In [6]:
# Run over everything
print("Running")
cerebro.run()
print("Done")

Running
2020-09-03, Close, 5.00
2020-09-03, BUY CREATE, 5.00
2020-09-04, Close, 2.06
2020-09-04, BUY CREATE, 2.06
2020-09-05, Close, 2.98
2020-09-06, Close, 2.72
2020-09-07, Close, 2.59
2020-09-07, BUY CREATE, 2.59
2020-09-08, Close, 2.86
2020-09-09, Close, 2.32
2020-09-10, Close, 2.71
2020-09-11, Close, 2.45
2020-09-12, Close, 2.24
2020-09-12, BUY CREATE, 2.24
2020-09-13, Close, 2.48
2020-09-14, Close, 2.00
2020-09-15, Close, 1.51
2020-09-15, BUY CREATE, 1.51
2020-09-16, Close, 1.39
2020-09-16, BUY CREATE, 1.39
2020-09-17, Close, 1.62
2020-09-18, Close, 1.86
2020-09-19, Close, 1.64
2020-09-20, Close, 1.45
2020-09-20, BUY CREATE, 1.45
2020-09-21, Close, 1.40
2020-09-21, BUY CREATE, 1.40
2020-09-22, Close, 1.29
2020-09-22, BUY CREATE, 1.29
2020-09-23, Close, 1.41
2020-09-24, Close, 1.39
2020-09-25, Close, 1.39
2020-09-25, BUY CREATE, 1.39
2020-09-26, Close, 1.39
2020-09-27, Close, 1.35
2020-09-28, Close, 1.23
2020-09-28, BUY CREATE, 1.23
2020-09-29, Close, 1.27
2020-09-30, Close, 1.26
2

/var/folders/wg/6tjsdnt511s90hqpxtxgkndc0000gn/T/ipykernel_27452/3073892212.py:5: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg')


## Plotting the results

In [7]:
# TODO: Displaying graphics from Backtrader in Jupyter notebook is broken
# See  https://github.com/enzoampil/fastquant/issues/382
#
# Returns two figures
# figures = cerebro.plot()
# figures[0][0]


<IPython.core.display.Javascript object>